In [ ]:
import numpy as np

# Load sudokus
sudoku = np.load("data/very_easy_puzzle.npy")
print("very_easy_puzzle.npy has been loaded into the variable sudoku")
print(f"sudoku.shape: {sudoku.shape}, sudoku[0].shape: {sudoku[0].shape}, sudoku.dtype: {sudoku.dtype}")

# Load solutions for demonstration
solutions = np.load("data/very_easy_solution.npy")
print()

# Print the first 9x9 sudoku...
print("First sudoku:")
print(sudoku[0], "\n")

# ...and its solution
print("Solution of first sudoku:")
print(solutions[0])

## Part One
You should write all of your code for solving sudokus below this cell.

You must include a function called `sudoku_solver(sudoku)` which takes one sudoku puzzle (a 9x9 NumPy array) as input, and returns the solved sudoku as another 9x9 NumPy array. This is the function which will be tested. 

In [ ]:
import numpy as np
from itertools import product 

# Checks the grid rows for repeating numbers
def validRows(grid):
  for row in grid:
    nonZero = [x for x in row if x != 0]
    if (len(nonZero) != len(set(nonZero))): return False
  return True

# Checks the columns for repeating numbers
def validColumns(grid):
  column = [[x[itr] for x in grid if x[itr] != 0] for itr in range(9) ]
  return False not in [True if len(set(c)) == len(c) else False for c in column]

# Checks grids for repeating nummbers 
def validBoxes(grid): 
  isValid = True
  for itr in range(9):
    if isValid == True: 
      flatList = np.array([grid[itr][0:3], grid[itr][3:6], grid[itr][6:9]]).flatten()
      flatList = flatList[flatList != 0]
      isValid = len(flatList) == len(set(flatList))
  return isValid
        
# Minimizes possible values through constraint programming
def constraintCheck(trialMatrix, row, col, trialValue):
    trialValue = trialValue - 1
    modifiedList = []
    for itr in range(9):
      if trialMatrix[itr][col][trialValue] == 1:
        modifiedList.append((itr, col))
        trialMatrix[itr][col][trialValue] = 0

    for itr in range(len(trialMatrix[row])):
      if trialMatrix[row][itr][trialValue] == 1:
        modifiedList.append((row,itr))
        trialMatrix[row][itr][trialValue] = 0

    for itr in product(range((row // 3)*3, (row // 3)*3 + 3), range((col // 3) * 3, (col // 3)*3 + 3)):
      if trialMatrix[itr[0]][itr[1]][trialValue] == 1:
          modifiedList.append((itr[0],itr[1]))
          trialMatrix[itr[0]][itr[1]][trialValue] = 0
  
    return modifiedList

# Carries out the backtracking depth first search
def search(grid, trialMatrix, rowValues, colValues, trialValues):
    
    for itr in product(range(len(grid)), range(len(grid[0]))):
      if grid[itr[0]][itr[1]] == 0 and sum(trialMatrix[itr[0]][itr[1]]) < trialValues:
        trialValues = sum(trialMatrix[itr[0]][itr[1]])
        rowValues = itr[0]
        colValues = itr[1]

    isValidCell = (rowValues, colValues) if trialValues < 10 else False    
        
    if isValidCell: row, col = isValidCell 
    else: return True

    for pValue in range(9):
        if trialMatrix[row][col][pValue] != 1: continue
        minSudoku = constraintCheck(trialMatrix, row, col, pValue +1)
        grid[row][col] = pValue + 1
        if search(grid, trialMatrix, -1 , -1, 10): return True
        # Updates the trial matrix, minimizing the problem space through constraint programming
        for row_x, col_x in minSudoku:
          trialMatrix[row_x][col_x][pValue] = 1
        grid[row][col] = 0

    return False # Unsolvable sudoku grid

# Checks for initial invalid sudokus then initialises a possible value space which is then solved
InvalidSudoku = np.full((9, 9), -1)
def sudoku_solver(sudoku):
    if not (validRows(sudoku) and validColumns(sudoku) and validBoxes(sudoku)): return InvalidSudoku 
    trialMatrix = np.ones((9,9,9))
    for itr in product(range(9), range(9)):
          if sudoku[itr[0]][itr[1]] != 0: constraintCheck(trialMatrix, itr[0], itr[1], sudoku[itr[0]][itr[1]]) 
    return sudoku if search(sudoku, trialMatrix,-1,-1, 10) else InvalidSudoku

All of your code must go above this cell. You may add additional cells into the notebook if you wish, but do not duplicate or copy/paste cells as this can interfere with the grading script.

### Testing Details
There are four difficulties of sudoku provided: very easy, easy, medium, and hard. There are 15 sample sudokus in each category, with solutions as well. Difficulty was determined using reference solvers, but your code may vary; it is conceivable that your code will find some sudokus much easier or harder within a given category, or even between categories.

*All categories that are easy and above will contain* ***invalid initial states***, that is, sudoku puzzles with no solution. In this case, your function should return a 9x9 NumPy array whose values are all equal to -1.

When we test your code, we will firstly test it on the *same* very easy puzzles that you have been given. Then we will test it on additional *hidden* sudokus from each difficulty in turn, easy and up – many more than 15 of each. Grades are awarded based on whether your code can solve the puzzles. For high grades on the hard puzzles, execution time will also be a factor. 

All puzzles must take under 20 seconds each on the test machine to count as successful – if it takes longer, it will timeout. Note that this is a maximum, not a goal. Higher grades require better performance on harder puzzles. As a very rough benchmark, you should be aiming for an average of under a second per puzzle. Hardware varies, but all tests will take place on the same modern desktop machine. Our ‘standard constraint satisfaction’ implementation takes about 0.001 seconds per puzzle for the very easy category, but struggles to solve some of the hard puzzles within the time limit.

***The hard sudokus are labelled as hard for a reason.*** We expect most submissions will not be able to solve them in a reasonable length of time. Use the stop button (■) on the toolbar if you need to terminate your code because it is taking too long.

The best way to improve the performance of your code is through a detailed understanding and smart choice of AI algorithms and data structures. This assignment is ***not*** meant to test your ability to write multi-threaded code or any other kind of high-performance code optimisations. 

#### Test Cell
The following code will run your solution over the provided sudoku puzzles. To enable it, set the constant `SKIP_TESTS` to `False`. If you fail any tests of one difficulty, the code will stop, but you can modify this behaviour if you like.

**IMPORTANT**: you must set `SKIP_TESTS` back to `True` before submitting this file!

In [ ]:
SKIP_TESTS = True

def tests():
    import time
    difficulties = ['very_easy','easy', 'medium','hard']

    for difficulty in difficulties:
        print(f"Testing {difficulty} sudokus")
        
        sudokus = np.load(f"data/{difficulty}_puzzle.npy")
        solutions = np.load(f"data/{difficulty}_solution.npy")
        
        count = 0
        for i in range(len(sudokus)):
            sudoku = sudokus[i].copy()
            print(f"This is {difficulty} sudoku number", i)
            print(sudoku)
            
            start_time = time.process_time()
            your_solution = sudoku_solver(sudoku)
            end_time = time.process_time()
            
            print(f"This is your solution for {difficulty} sudoku number", i)
            print(your_solution)
            
            print("Is your solution correct?")
            if np.array_equal(your_solution, solutions[i]):
                print("Yes! Correct solution.")
                count += 1
            else:
                print(i)
                print("No, the correct solution is:")
                print(solutions[i])
            
            print("This sudoku took", end_time-start_time, "seconds to solve.\n")

        print(f"{count}/{len(sudokus)} {difficulty} sudokus correct")
        if count < len(sudokus):
            break
            
if not SKIP_TESTS:
    tests()

## Submission Test
The following cell tests if your notebook is ready for submission. **You must not skip this step!**

Restart the kernel and run the entire notebook (Kernel → Restart & Run All). Now look at the output of the cell below. 

*If there is no output, then your submission is not ready.* Either your code is still running (did you forget to skip tests?) or it caused an error.

As previously mentioned, failing to follow these instructions can result in a grade of zero.

In [ ]:
def submission_tests():
    import sys
    import pathlib

    fail = False;

    if not SKIP_TESTS:
        fail = True;
        print("You must set the SKIP_TESTS constant to True in the cell above.")

    p1 = pathlib.Path('./readme.txt')
    p2 = pathlib.Path('./readme.md')
    if not (p1.is_file() or p2.is_file()):
        fail = True;
        print("You must include a separate file called readme.txt or readme.md in your submission.")

    p3 = pathlib.Path('./sudoku.ipynb')
    if not p3.is_file():
        fail = True
        print("This notebook file must be named sudoku.ipynb")

    if "sudoku_solver" not in globals():
        fail = True;
        print("You must include a function called sudoku_solver which accepts a numpy array.")
    else: 
        sudoku = np.load("data/very_easy_puzzle.npy")[0]
        solution = np.load("data/very_easy_solution.npy")[0]

        if not np.array_equal(sudoku_solver(sudoku), solution):
            print("Warning:")
            print("Your sudoku_solver function does not correctly solve the first sudoku.")
            print()
            print("Your assignment is unlikely to get any marks from the autograder. While we will")
            print("try to check it manually to assign some partial credit, we encourage you to ask")
            print("for help on the forum or directly to a tutor.")
            print()
            print("Please use the readme file to explain your code anyway.")

    if fail:
        print()
        sys.stderr.write("Your submission is not ready! Please read and follow the instructions above.")
    else:
        print("All checks passed. When you are ready to submit, upload the notebook and readme file to the")
        print("assignment page, without changing any filenames.")
        print()
        print("If you need to submit multiple files, you can archive them in a .zip file. (No other format.)")
        
submission_tests()

In [ ]:
# This is a TEST CELL. Do not delete or change.